In [6]:
import torch
import rasterio

# Define the directory path where the rasters are located
directory = '/nfs/home/innocenti/igarss-fmars/data/ithaca-val/gt_26'
# Get a list of all raster files in the directory
raster_files = rasterio.os.listdir(directory)
# sort the list
raster_files.sort()
# Create an empty list to store the loaded rasters
labels = []
# Iterate over each raster file
for file in raster_files:
    # Open the raster file using rasterio
    with rasterio.open(directory + '/' + file) as src:
        # Read the raster data
        raster_data = src.read(1)  # Change the band number if needed
        # shift all labels by 1, set 255 to 1
        raster_data = raster_data + 1
        raster_data[raster_data == 256] = 0
        # Convert the raster data to a torch tensor
        labels.append(torch.from_numpy(raster_data))
        

In [7]:
print(len(labels))
print(labels[0].shape)
print(labels[0].unique())

26
torch.Size([512, 512])
tensor([0, 1, 2, 3], dtype=torch.uint8)


In [8]:
import torch
import rasterio

# Define the directory path where the rasters are located
directory = '/nfs/home/innocenti/igarss-fmars/output_images/segformer_val_05'
# Get a list of all raster files in the directory
raster_files = rasterio.os.listdir(directory)
# sort the list
raster_files.sort()
# Create an empty list to store the loaded rasters
inferences = []
# Iterate over each raster file
for file in raster_files:
    # Open the raster file using rasterio
    with rasterio.open(directory + '/' + file) as src:
        # Read the raster data
        raster_data = src.read(1)  # Change the band number if needed
        
        # Convert the raster data to a torch tensor
        inferences.append(torch.from_numpy(raster_data))

/nfs/home/innocenti/igarss-fmars/.venv2/lib/python3.8/site-packages/rasterio/__init__.py:304: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


In [9]:
print(len(inferences))
print(inferences[0].shape)
print(inferences[0].unique())

26
torch.Size([512, 512])
tensor([0, 2], dtype=torch.uint8)


In [10]:
from torchmetrics import Accuracy, JaccardIndex

# Convert the labels and inferences to torch tensors
labels_tensor = torch.stack(labels)
inferences_tensor = torch.stack(inferences)

# Create the accuracy metric
accuracy_perclass = Accuracy(num_classes=4, task='multiclass', average='none')
accuracy_micro = Accuracy(num_classes=4, task='multiclass', average='micro')
accuracy_macro = Accuracy(num_classes=4, task='multiclass', average='macro')
accuracy_ignorebg = Accuracy(num_classes=4, task='multiclass', average='micro', ignore_index=0)

# Compute the accuracy between labels and inferences
accuracy_perclass_value = accuracy_perclass(inferences_tensor, labels_tensor)
accuracy_micro_value = accuracy_micro(inferences_tensor, labels_tensor)
accuracy_macro_value = accuracy_macro(inferences_tensor, labels_tensor)
accuracy_ignorebg_value = accuracy_ignorebg(inferences_tensor, labels_tensor)

# Create the IoU metric
iou_perclass = JaccardIndex(num_classes=4, task='multiclass', average='none')
iou_micro = JaccardIndex(num_classes=4, task='multiclass', average='micro')
iou_macro = JaccardIndex(num_classes=4, task='multiclass', average='macro') 
iou_ignorebg = JaccardIndex(num_classes=4, task='multiclass', average='micro', ignore_index=0)

# Compute the mIoU between labels and inferences
iou_perclass_value = iou_perclass(inferences_tensor, labels_tensor)
iou_micro_value = iou_micro(inferences_tensor, labels_tensor)
iou_macro_value = iou_macro(inferences_tensor, labels_tensor)
iou_ignorebg_value = iou_ignorebg(inferences_tensor, labels_tensor)

# Print the accuracy and mIoU values
print('Accuracy per class:', accuracy_perclass_value)
print('Accuracy micro:', accuracy_micro_value)
print('Accuracy macro:', accuracy_macro_value)
print('Accuracy ignore bg:', accuracy_ignorebg_value)
print('IoU per class:', iou_perclass_value)
print('IoU micro:', iou_micro_value)
print('IoU macro:', iou_macro_value)
print('IoU ignore bg:', iou_ignorebg_value)

Accuracy per class: tensor([9.7447e-01, 5.8893e-04, 8.2359e-02, 0.0000e+00])
Accuracy micro: tensor(0.2897)
Accuracy macro: tensor(0.2644)
Accuracy ignore bg: tensor(0.0256)
IoU per class: tensor([0.2790, 0.0006, 0.0768, 0.0000])
IoU micro: tensor(0.1694)
IoU macro: tensor(0.0891)
IoU ignore bg: tensor(0.0253)
